# Fast Net GP

In [1]:
import fastgps
import qmcpy as qp
import torch
import numpy as np

In [2]:
device = "cpu"
if device!="mps":
    torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f_ackley(x, a=20, b=0.2, c=2*np.pi, scaling=32.768):
    # https://www.sfu.ca/~ssurjano/ackley.html
    assert x.ndim==2
    x = 2*scaling*x-scaling
    t1 = a*torch.exp(-b*torch.sqrt(torch.mean(x**2,1)))
    t2 = torch.exp(torch.mean(torch.cos(c*x),1))
    t3 = a+np.exp(1)
    y = -t1-t2+t3
    return y
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng).to(device) # random testing locations
y = f_ackley(x) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng).to(device) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (128,)
z.shape = (256, 1)


## Construct Fast GP

In [4]:
fgp = fastgps.FastGPDigitalNetB2(qp.KernelDigShiftInvar(d,torchify=True,device=device),seqs=7)
x_next = fgp.get_x_next(2**10)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
print("x_next.shape = %s"%str(tuple(x_next.shape)))
print("y_next.shape = %s"%str(tuple(y_next.shape)))

x_next.shape = (1024, 1)
y_next.shape = (1024,)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error = %.2e"%(torch.linalg.norm(y-pmean)/torch.linalg.norm(y)))

pmean.shape = (128,)
l2 relative error = 8.37e-03


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | best loss  | loss       | term1      | term2     
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | 1.11e+04   | 1.11e+04   | 2.44e+04   | -4.01e+03 
            5.00e+00 | 2.49e+03   | 2.49e+03   | 5.59e+03   | -2.48e+03 
            1.00e+01 | 1.10e+03   | 1.12e+03   | 1.35e+03   | -9.97e+02 
            1.50e+01 | 1.10e+03   | 1.10e+03   | 1.02e+03   | -7.03e+02 
            2.00e+01 | 1.10e+03   | 1.10e+03   | 1.01e+03   | -6.99e+02 
            2.50e+01 | 1.09e+03   | 1.09e+03   | 1.02e+03   | -7.18e+02 
            3.00e+01 | 1.09e+03   | 1.09e+03   | 1.02e+03   | -7.22e+02 
            3.50e+01 | 1.09e+03   | 1.09e+03   | 1.07e+03   | -7.75e+02 
            4.00e+01 | 1.09e+03   | 1.09e+03   | 1.07e+03   | -7.74e+02 
            4.50e+01 | 1.08e+03   | 1.08e+03   | 1.06e+03   | -7.74e+02 
            5.00e+01 | 1.08e+03   | 1.08e+03   | 1.04e+03   | -7.74e+02 
            5.50e+01 | 1.07e+03   | 1.07e+03   | 1.

[]

In [7]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error = %.2e"%(torch.linalg.norm(y-pmean)/torch.linalg.norm(y)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))
print("\npcov diag matches pvar: %s"%torch.allclose(pcov.diagonal(),pvar))
print("non-negative pvar: %s"%(pvar>=0).all().item())

pmean.shape = (128,)
pvar.shape = (128,)
q = 2.58
ci_low.shape = (128,)
ci_high.shape = (128,)
l2 relative error = 8.43e-03
pcov.shape = (128, 128)
pcov2.shape = (128, 256)

pcov diag matches pvar: True
non-negative pvar: True


In [8]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean = %.3e"%pcmean)
print("pcvar = %.3e"%pcvar)
print("cci_low = %.3e"%cci_low)
print("cci_high = %.3e"%cci_high)

pcmean = 1.841e+01
pcvar = 2.298e-02
cci_low = 1.802e+01
cci_high = 1.880e+01


## Project and Increase Sample Size

In [9]:
pcov_future = fgp.post_cov(x,z,n=2*fgp.n)
pvar_future = fgp.post_var(x,n=2*fgp.n)
pcvar_future = fgp.post_cubature_var(n=2*fgp.n)

In [10]:
x_next = fgp.get_x_next(2*fgp.n)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
print("l2 relative error = %.2e"%(torch.linalg.norm(y-fgp.post_mean(x))/torch.linalg.norm(y)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

l2 relative error = 4.73e-03


In [11]:
data = fgp.fit(verbose=False)
print("l2 relative error = %.2e"%(torch.linalg.norm(y-fgp.post_mean(x))/torch.linalg.norm(y)))

l2 relative error = 4.73e-03


In [12]:
pcov_8n = fgp.post_cov(x,z,n=8*fgp.n)
pvar_8n = fgp.post_var(x,n=8*fgp.n)
pcvar_8n = fgp.post_cubature_var(n=8*fgp.n)
x_next = fgp.get_x_next(8*fgp.n)
y_next = f_ackley(x_next)
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_8n)
assert torch.allclose(fgp.post_var(x),pvar_8n)
assert torch.allclose(fgp.post_cubature_var(),pcvar_8n)